### Entyty-Relationship Diagram

In [ ]:


from IPython.display import Image
Image(filename="C:/Users/Acer/OneDrive/Зображення/ERD_e-Comm(target).png")

# **Sales & Revenue & Payments Analysis**

## 1. Product Category Performance

### *Which product categories have the highest profit margins?*

In [ ]:
## total revenue from product prices and total shipping costs (freight_value) grouped by product categories.

import mysql.connector
import pandas as pd

# Connection to MySQL
conn = mysql.connector.connect(
    host='localhost',
    user='codebind',  
    password='58623',  
    database='mydb1'  
)


cursor = conn.cursor()

query = '''
SELECT 
    p.product_category_name AS category,
    SUM(order_items.price) AS total_revenue,
    SUM(order_items.freight_value) AS total_shipping_cost,
    SUM(order_items.price) - SUM(order_items.freight_value) AS profit_margin,
    (SUM(order_items.price) - SUM(order_items.freight_value)) / SUM(order_items.price) * 100 AS profit_margin_percentage
FROM 
    order_items 
JOIN 
    products p ON order_items.product_id = p.product_id
GROUP BY 
    p.product_category_name
ORDER BY 
    profit_margin_percentage DESC;

'''

margin_df = pd.read_sql(query,conn)
margin_df

In [ ]:
#  percentiles for profit margins to set a ref line visualizing  data in Tableu
percentile_25 = margin_df['profit_margin_percentage'].quantile(0.25)
percentile_50 = margin_df['profit_margin_percentage'].median()
percentile_75 = margin_df['profit_margin_percentage'].quantile(0.75)

print("25th Percentile:", percentile_25)
print("50th Percentile (Median):", percentile_50)
print("75th Percentile:", percentile_75)


In [3]:
top_10 = margin_df.sort_values(by='profit_margin_percentage', ascending=False).head(10)
bottom_10 = margin_df.sort_values(by='profit_margin_percentage', ascending=True).head(10)
combined_df = pd.concat([top_10, bottom_10])
combined_df.to_csv('combined_margin.csv', index=False)



In [ ]:
from IPython.display import Image
Image(filename="C:/e-Commerce (Target) Sales/assets/highest_profit_margin.png")

## 2. Product Revenue Contribution and Volume Analysis

*WHY? TO Identify the products driving revenue and the impact of volume vs. price*

*What percentage of revenue comes from the top 20% of products (Pareto Principle)?*

In [ ]:
query = '''
SELECT 
    products.product_id, 
    SUM(order_items.price) AS total_sum, 
    products.product_category_name AS category
FROM 
    order_items
JOIN 
    products ON order_items.product_id = products.product_id
GROUP BY 
    products.product_id, products.product_category_name
ORDER BY 
    total_sum DESC;

'''

revenue_df = pd.read_sql(query,conn)
revenue_df

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

revenue_df['cumulative_revenue'] = revenue_df['total_sum'].cumsum()
revenue_df['cumulative_revenue_percentage'] = (revenue_df['cumulative_revenue'] / revenue_df['total_sum'].sum()) * 100

# Calculate cumulative product percentage
revenue_df['cumulative_products_percentage'] = (revenue_df.index + 1) / len(revenue_df) * 100

# Find the percentage of products that contribute to 80% revenue
top_80_percent = revenue_df[revenue_df['cumulative_revenue_percentage'] <= 80]

print(f"Top 20% Products Contribution to Total Revenue: {len(top_80_percent)} / {len(revenue_df)}")


In [ ]:
# Export DataFrame to a CSV file
revenue_df.to_csv('cumulative_revenue_data.csv', index=False)
Image(filename="C:/e-Commerce (Target) Sales/assets/pareto.png")


### *Which products generate high revenue but have low order volume?*

WHY? to find out expensive, low-order volume products that require targeted promotion

In [ ]:
import pandas as pd

order_items_df = pd.read_sql('SELECT product_id, price, order_id FROM order_items', conn)
products_df = pd.read_sql('SELECT product_id, product_category_name FROM products', conn)
merged_df = order_items_df.merge(products_df, on='product_id')





In [46]:
product_metrics = merged_df.groupby(['product_id', 'product_category_name']).agg(
    total_revenue=('price', 'sum'),
    total_orders=('order_id', 'count'),
    avg_revenue_per_order=('price', 'mean')
).reset_index()



In [52]:


# High order volume but low revenue per order
frequent_low_value = product_metrics[
    (product_metrics['total_orders'] > product_metrics['total_orders'].quantile(0.75)) &
    (product_metrics['avg_revenue_per_order'] < product_metrics['avg_revenue_per_order'].mean())
]



In [ ]:

print("\nFrequently Purchased Products with Low Revenue Per Order:")
print(frequent_low_value[['product_category_name', 'total_orders', 'avg_revenue_per_order']])

In [ ]:

expensive_low_orders = product_metrics[
    (product_metrics['total_revenue'] > product_metrics['total_revenue'].quantile(0.75)) &
    (product_metrics['total_orders'] < 10)  # Products with fewer than 10 orders
]

print("Expensive Products with Low Orders:")
print(expensive_low_orders[['product_category_name', 'total_revenue', 'total_orders']])



In [ ]:
import pandas as pd

# Aggregate and analyze categories in expensive_low_orders
expensive_category_analysis = expensive_low_orders.groupby('product_category_name').agg(
    total_revenue=('total_revenue', 'sum'),
    total_products=('product_category_name', 'count')
).reset_index().sort_values(by='total_products', ascending=False)

frequent_low_value_analysis = frequent_low_value.groupby('product_category_name').agg(
    total_orders=('total_orders', 'sum'),
    avg_revenue_per_order=('avg_revenue_per_order', 'mean'),
    total_products=('product_category_name', 'count')
).reset_index().sort_values(by='total_products', ascending=False)

print("Dominant Categories in Expensive Products with Low Orders:")
print(expensive_category_analysis.head(10))

print("\nDominant Categories in Frequently Purchased Products with Low Revenue Per Order:")
print(frequent_low_value_analysis.head(10))


In [ ]:
Image(filename="assets/comparison.png")

## 3. Payment Preferences and Revenue Contribution

WHY? In order to analyze which payment methods dominate revenue and how installments contribute

#*What are the most common payment methods?*

In [63]:
query = '''
SELECT 
    payment_type, 
    COUNT(payment_type) AS payments_count,
    SUM(payment_value) AS total_payment_value
FROM payments
GROUP BY payment_type
ORDER BY total_payment_value
'''

payment_df = pd.read_sql(query,conn)
payment_df

C:\Users\Acer\AppData\Local\Temp\ipykernel_3664\364083006.py:11: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  payment_df = pd.read_sql(query,conn)


,payment_type,payments_count,total_payment_value
0,not_defined,6,0.000000e+00
1,debit_card,3058,4.359796e+05
2,voucher,11550,7.588737e+05
3,UPI,39568,5.738723e+06
4,credit_card,153590,2.508417e+07


In [65]:
import pandas as pd

# Correctly formatting and exporting the DataFrame
export_df = payment_df[['payment_type', 'payments_count', 'total_payment_value']]

# Exporting with a proper comma separator and headers
export_df.to_csv('payment_analysis_corrected.csv', index=False)

print("Exported successfully to 'payment_analysis_corrected.csv'!")



Exported successfully to 'payment_analysis_corrected.csv'!
